# ANN
#Predict the electrical energy output for the
#Combined Cycle Power Plant Data Set


*Adhiraj Roka*

### Importing the packages

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
!pip install -q xlrd


### Importing the dataset

Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Setting up directory

In [ ]:
%cd /content/gdrive/My\ Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


Importing the dataset

In [ ]:
energy = pd.read_excel('Folds5x2_pp.xlsx')
X=energy.iloc[:,0:4].values
Y=energy.iloc[:,-1].values

In [ ]:
print(energy)
print(X)
print(Y)

         AT      V       AP     RH      PE
0     14.96  41.76  1024.07  73.17  463.26
1     25.18  62.96  1020.04  59.08  444.37
2      5.11  39.40  1012.16  92.14  488.56
3     20.86  57.32  1010.24  76.64  446.48
4     10.82  37.50  1009.23  96.62  473.90
...     ...    ...      ...    ...     ...
9563  16.65  49.69  1014.01  91.00  460.03
9564  13.19  39.18  1023.67  66.78  469.62
9565  31.32  74.33  1012.92  36.48  429.57
9566  24.48  69.45  1013.86  62.39  435.74
9567  21.60  62.52  1017.23  67.87  453.28

[9568 rows x 5 columns]
[[  14.96   41.76 1024.07   73.17]
 [  25.18   62.96 1020.04   59.08]
 [   5.11   39.4  1012.16   92.14]
 ...
 [  31.32   74.33 1012.92   36.48]
 [  24.48   69.45 1013.86   62.39]
 [  21.6    62.52 1017.23   67.87]]
[463.26 444.37 488.56 ... 429.57 435.74 453.28]


### Splitting the dataset into the Training set and Test 



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, 
                                                    random_state = 0) 
#Target is continuous so no need to stratify

### Feature Scaling

In [ ]:
#Apply standardization to all inputs#

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#print(X_train)

## Building the ANN

### Initializing the ANN

In [ ]:
tf.random.set_seed(123)
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=3, activation='relu'))

### Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=3, activation='relu'))

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='relu')) 
#can also just use "linear" activation function. Relu gave better mse.

### Compiling the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mse','mae'])

### Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, validation_data= (X_test, y_test),  batch_size = 32, 
        epochs = 120)

Epoch 1/120
240/240 [==============================] - 0s 1ms/step - loss: 206233.5000 - mse: 206233.5000 - mae: 453.7996 - val_loss: 205651.7656 - val_mse: 205651.7656 - val_mae: 453.1364
Epoch 2/120
240/240 [==============================] - 0s 1ms/step - loss: 202539.6875 - mse: 202539.6875 - mae: 449.6234 - val_loss: 198312.2656 - val_mse: 198312.2656 - val_mae: 444.8155
Epoch 3/120
240/240 [==============================] - 0s 987us/step - loss: 189076.4375 - mse: 189076.4375 - mae: 434.0835 - val_loss: 178059.5000 - val_mse: 178059.5000 - val_mae: 421.0169
Epoch 4/120
240/240 [==============================] - 0s 993us/step - loss: 160652.8750 - mse: 160652.8750 - mae: 398.8146 - val_loss: 141092.8438 - val_mse: 141092.8438 - val_mae: 372.5876
Epoch 5/120
240/240 [==============================] - 0s 1ms/step - loss: 118971.9453 - mse: 118971.9453 - mae: 338.3445 - val_loss: 97760.6797 - val_mse: 97760.6797 - val_mae: 303.1222
Epoch 6/120
240/240 [==============================] 

In [ ]:
ann.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 3)                 15        
_________________________________________________________________
dense_32 (Dense)             (None, 3)                 12        
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 4         
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________


## Calculating MSE, MAE and making the predictions for Sheet 2

In [ ]:
y_pred = ann.predict(X_test)
print(y_pred)
print(y_test)
#print(np.concatenate((y_pred, y_test.reshape(len(y_test),1)),1))

[[430.54395]
 [458.36487]
 [462.87994]
 ...
 [469.53876]
 [441.64746]
 [461.63715]]
[431.23 460.01 461.14 ... 473.26 438.   463.28]


In [ ]:
#MSE and MAE#

from sklearn.metrics import mean_squared_error, mean_absolute_error
print(mean_squared_error(y_test, y_pred))#test mse
print(mean_absolute_error(y_test,y_pred))#test mae

19.582243048915974
3.5256163708917025


*For the final model*,

**Training** set: **mse**: 21.1644 , **mae**: 3.6456

**Testing** set: **mse**: 19.5822, **mae**: 3.5256

##Now make prediction for Sheet 2


In [ ]:
#Run on SHEET 2#
sheet2= pd.read_excel('Folds5x2_pp.xlsx',sheet_name=1)
print(sheet2)
#Separate into input X,Y vars fromm sheet2
X_sheet=sheet2.iloc[:,0:-1].values
Y_sheet=sheet2.iloc[:,-1].values

pred=sc.transform(X_sheet)#apply feature scaling on all input variables in Sheet 2
sheet2_y_pred=ann.predict(pred) #predict y in sheet 2


print(sheet2_y_pred)
print(Y_sheet)

#Concatenate prediction with actual values#
print(np.concatenate((sheet2_y_pred, Y_sheet.reshape(len(Y_sheet),1)),1))

#MSE for the sheet 2 predicted vs actual
print(mean_squared_error(sheet2_y_pred, Y_sheet)) 


         AT      V       AP     RH      PE
0      9.59  38.56  1017.01  60.10  481.30
1     12.04  42.34  1019.72  94.67  465.36
2     13.87  45.08  1024.42  81.69  465.48
3     13.72  54.30  1017.89  79.08  467.05
4     15.14  49.64  1023.78  75.00  463.58
...     ...    ...      ...    ...     ...
9563  17.10  49.69  1005.53  81.82  457.32
9564  24.73  65.34  1015.42  52.80  446.92
9565  30.44  56.24  1005.19  56.24  429.34
9566  23.00  66.05  1020.61  80.29  421.57
9567  17.75  49.25  1020.86  63.67  454.41

[9568 rows x 5 columns]
[[480.0119 ]
 [469.38785]
 [467.22083]
 ...
 [433.84286]
 [443.82776]
 [460.81982]]
[481.3  465.36 465.48 ... 429.34 421.57 454.41]
[[480.01190186 481.3       ]
 [469.3878479  465.36      ]
 [467.2208252  465.48      ]
 ...
 [433.84286499 429.34      ]
 [443.82775879 421.57      ]
 [460.81982422 454.41      ]]
20.772329908675957
